### Building a RAG System with LangChain and FAISS 
Introduction to RAG (Retrieval-Augmented Generation)
RAG combines the power of retrieval systems with generative AI models. Instead of relying solely on the model's training data, RAG:

1. Retrieves relevant documents from a knowledge base
2. Uses these documents as context for the LLM
3. Generates responses based on both the retrieved context and the model's knowledge

### FAISS 
https://github.com/facebookresearch/faiss

FAISS is a library for efficient similarity search and clustering of dense vectors.

Key advantages:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

How it works:
- Indexes vectors for fast nearest neighbor search
- Returns most similar vectors based on distance metrics


In [143]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough, 
 
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Load environment variables
load_dotenv()

True

### Data Ingestion And Processing


In [144]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [145]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])


page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [146]:

print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [147]:
### load the embedding models
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [148]:
# Initialize OpenAI embeddings with the latest model

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)

## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding=embeddings.embed_query(sample_text)
sample_embedding

[-0.005943009164184332,
 -0.005915995221585035,
 0.0005588454660028219,
 -0.03358340263366699,
 0.02124355547130108,
 0.02211879938840866,
 -0.0007665131124667823,
 0.009298107586801052,
 -0.022583434358239174,
 0.03781914710998535,
 0.015246519818902016,
 -0.03667376935482025,
 -0.03399401158094406,
 0.001597859081812203,
 0.026711124926805496,
 0.01601370796561241,
 0.0008036569342948496,
 -0.030860425904393196,
 0.02336142770946026,
 0.004905683919787407,
 5.145261457073502e-05,
 0.03248124569654465,
 0.04220616817474365,
 -0.029326047748327255,
 0.015927264466881752,
 -0.020951807498931885,
 0.026646291837096214,
 0.012393875047564507,
 0.0074503724463284016,
 -0.005986230913549662,
 -0.009470995515584946,
 -0.0290451068431139,
 -0.03133586421608925,
 0.027553951367735863,
 0.04170911759138107,
 0.0004153352929279208,
 -0.00805547833442688,
 -0.010659596882760525,
 -0.05554012209177017,
 0.002604118548333645,
 -0.0562748946249485,
 -0.016251428052783012,
 0.030276929959654808,
 0.0

In [149]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)
print(batch_embeddings[0])

[-0.008208601735532284, -0.024612585082650185, 0.002946041990071535, 0.025167757645249367, 0.006533174309879541, -0.02826085314154625, -0.0050229765474796295, 0.020977536216378212, -0.036879222840070724, 0.012868072837591171, -0.0030385705176740885, -0.020144781097769737, 0.000270769844064489, -0.032781533896923065, 0.006414209026843309, -0.02529994025826454, -0.031010271981358528, -0.05440676957368851, 0.03280796855688095, -0.01842639409005642, 0.01658904179930687, 0.04832632467150688, -0.024876952171325684, 0.014341920614242554, 0.0293976329267025, 0.004044818226248026, 0.009246242232620716, 0.0133439339697361, 0.0025065315421670675, -0.02259017713367939, 0.03214704990386963, -0.028022922575473785, 0.005317085422575474, -0.038227494806051254, -0.0167080070823431, 0.014355138875544071, -0.03859760984778404, -0.01037641242146492, -0.01056146901100874, -0.01924593187868595, 0.03206774219870567, 0.014632724225521088, -0.02155914530158043, 0.016126397997140884, -0.011843650601804256, 0.00

In [150]:
print(batch_embeddings[1])

[-0.01823362149298191, 0.01074262149631977, 0.017460593953728676, -0.03524021804332733, 0.037424325942993164, 0.02608659490942955, 0.015890000388026237, 0.008374459110200405, -0.010945080779492855, 0.034872107207775116, -0.006638216320425272, -0.06866443157196045, -0.0323689728975296, 0.008969567716121674, 0.03658994659781456, 0.018712162971496582, 0.016270378604531288, -0.015399189665913582, 0.020614054054021835, 0.012945135124027729, -0.00261049997061491, 0.03781697154045105, 0.03403773158788681, -0.030013080686330795, 0.020650865510106087, 0.007926594465970993, 0.03614821657538414, 0.03766972944140434, 0.01052175648510456, -0.03482302650809288, -0.010932810604572296, -0.024233784526586533, -0.013914486393332481, -0.013865405693650246, 0.011945107951760292, -0.0018451418727636337, -0.027583567425608635, 0.027828972786664963, -0.04508097469806671, -0.005190324503928423, -0.019755134359002113, -0.04277416318655014, 0.03438129648566246, 0.06827178597450256, -0.020577242597937584, -0.034

In [151]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [152]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [153]:
print(f"'AI' vs 'Pizza': {compare_embeddings('AI', 'Pizza'):.3f}")

'AI' vs 'Pizza': 0.254


In [154]:
print(f"'Machine Learning' vs 'ML': {compare_embeddings('Machine Learning', 'ML'):.3f}")

'Machine Learning' vs 'ML': 0.461


### Create FAISS Vector Store

In [155]:
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [156]:
vectorstore

In [157]:
## Save vector tore for later use
vectorstore.save_local("faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [158]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [159]:
## Similarity Search 
query="What is deep learning"

results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='02f0b0c9-c617-4b09-92d4-c669edeeeb1d', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='e0478927-1e00-4169-a00e-23e69450b4f2', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='4a212780-764a-436a-a346-c4dd343fa4a8', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [160]:
print(f"Query: {query}\n")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning

Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [161]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nSimilarity search with scores:")
for doc, score in results_with_scores:
    print(f"\nScore: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")



Similarity search with scores:

Score: 0.556
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.208
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.274
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [162]:
chunks

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

In [163]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='e0478927-1e00-4169-a00e-23e69450b4f2', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [164]:
len(filtered_results)

1

### Build RAG Chain With LCEL 

In [165]:
## LLM GROQ LLM
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F1E5C330A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F1E5BB4940>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [166]:
llm.invoke("Hi")

BadRequestError: Error code: 400 - {'error': {'message': 'The model `gemma2-9b-it` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

In [ ]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [ ]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [ ]:
retriever

In [ ]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [ ]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | llm
    |StrOutputParser()

)

In [ ]:
simple_rag_chain

In [ ]:
### Conversational RAg Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [ ]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [ ]:
conversational_rag

In [ ]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

In [ ]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [ ]:
test_rag_chains("What is the difference between AI and machine learning")

In [ ]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)

In [ ]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")

In [ ]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [ ]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")